In [7]:
import requests

from datetime import date, datetime

In [10]:
# datetime object containing current date and time
now = datetime.now()

dthr_finalizado = now.strftime("%d/%m/%Y %H:%M:%S")
print("date and time =", dthr_finalizado)	

date and time = 12/10/2022 09:30:28


In [6]:

# By using a context manager, you can ensure the resources used by
# the session will be released after use
with requests.Session() as session:
    #num_proj = int(listagem['Projeto'][i])
    #ano_proj = int(listagem['Ano'][i])
    #tipo_proj = listagem['Codigo_Tipo'][i]
    
    query = {"numero": 193600, "ano": 2012, "tipo": 'REQ'}
        
    try:
        # Requisicao para o portal de propostas da Camara
        #query = {"numero": num_proj, "ano": ano_proj, "codtipo": tipo_proj}
        resposta = session.get('https://www.camarajf.mg.gov.br/sal/listapropos.php?', params=query)
        print(resposta.headers)
        #print(resposta.json())
        print(resposta.status_code)
    except:
        print('Falhou')
    session.close()
# You can inspect the response just like you did before


{'Date': 'Tue, 11 Oct 2022 10:33:02 GMT', 'Server': 'Apache', 'X-Frame-Options': 'SAMEORIGIN', 'X-Content-Type-Options': 'nosniff', 'X-XSS-Protection': '1; mode=block', 'X-Powered-By': 'PHP/7.1.18', 'Vary': 'Accept-Encoding', 'Content-Encoding': 'gzip', 'Content-Length': '1497', 'Keep-Alive': 'timeout=5, max=100', 'Connection': 'Keep-Alive', 'Content-Type': 'text/html'}
200


### Novo (session)

In [ ]:
# Preenchimento do campo Id_Projeto
count = 1
failed_req= 0
failed = False
id_projeto = ''
max_failed_req = 5
notfound_count = 0

while count > 0:
    for i in listagem.index:
        if listagem['Id_Projeto'][i] == '' and listagem['Codigo_Tipo'][i] not in ('DESCONHECIDO','EMENDA'): 
            with requests.Session() as session:
                id_projeto = ''
                failed = False
                # Variaveis de requisicao
                num_proj = int(listagem['Projeto'][i])
                ano_proj = int(listagem['Ano'][i])
                tipo_proj = listagem['Codigo_Tipo'][i]

                try:
                    # Requisicao para o portal de propostas da Camara
                    query = {"numero": num_proj, "ano": ano_proj, "codtipo": tipo_proj}
                    #query = {"numero": 193600, "ano": 2012, "tipo": 'REQ'}
                    resposta = requests.get('https://www.camarajf.mg.gov.br/sal/listapropos.php?', params=query)
                    try:
                        # Limpeza para pegar o id do projeto dentro da prefeitura
                        retorno = (''.join(map(chr, resposta.content.upper())).replace(' ', ''))
                        id_projeto = re.search('PROPOSICAO.PHP(.+?)TARGET=BLANK', retorno).group(1).replace('?NUM=','').replace('"','')
                    except:
                        try:
                            retorno = (''.join(map(chr, resposta.content.upper())).replace(' ', ''))
                            id_projeto = re.search('PROPOSICAON.PHP(.+?)TARGET=BLANK', retorno).group(1).replace('?NUM=','').replace('"','')
                        except:
                            id_projeto = 'Nao Encontrado'
                            notfound_count += 1
                except:
                    #Assinala que houve falha
                    failed = True
                    time.sleep(180)

                listagem['Id_Projeto'][i] = id_projeto
                resposta.close()
                session.close()
                time.sleep(0.5)
        if failed:
            failed_req += 1
        else:
            filed_req = 0
                
#       Se houve 5 falhas consecutivas
        if failed_req > max_failed_req:
            break

    if failed_req > max_failed_req:
        break   
        
    count = 0
    
    for i in listagem.index:
        if listagem['Id_Projeto'][i] == '' and listagem['Codigo_Tipo'][i] not in ('DESCONHECIDO','EMENDA'):
            count +=1

if failed_req > max_failed_req:
    print(f'O endpoint nao esta respondendo, verifique a conexao. Fim do processo. Nao encontrados = {notfound_count}')
else:
    print('Fim do processo')

In [ ]:
# Preenchimento do campo Data_Projeto, Tramitacao_Encerrada e Aprovado
count = 1
failed_req= 0
failed = False
max_failed_req = 5
notfound_count = 0

data_projeto = ''
notfound_count_dt = 0

tramitacao_encerrada = ''
notfound_count_tram = 0

aprovado = ''
notfound_count_aprv = 0

while count > 0:
    for i in listagem.index:
        if (listagem['Data_Projeto'][i] == '' or listagem['Tramitacao_Encerrada'][i] == '' or listagem['Aprovado'][i] == '') and listagem['Id_Projeto'][i] != '': 
            with requests.Session() as session:
                data_projeto = ''
                tramitacao_encerrada = ''
                aprovado = ''

                failed = False
                # Variaveis de requisicao
                id_proj = listagem['Id_Projeto'][i]

                try:
                    # Requisicao para o portal de propostas da Camara
                    query = {"num": id_proj}
                    resposta = requests.get('https://www.camarajf.mg.gov.br/sal/proposicao.php?', params=query)
                    retorno = (''.join(map(chr, resposta.content.upper())).replace(' ', ''))

                    #Identificacao da data de registro do projeto
                    try:
                        # Limpeza para pegar o id do projeto dentro da prefeitura
                        #retorno = (''.join(map(chr, resposta.content.upper())).replace(' ', ''))
                        data_projeto = re.search('PER&IACUTE;ODO:(.+?)A', retorno).group(1)
                    except:
                        data_projeto = 'Data nao encontrada'
                        notfound_count_dt += 1


                    #Identificacao do encerramento da tramitacao do projeto
                    try:
                        # Limpeza para pegar o id do projeto dentro da prefeitura
                        #retorno = (''.join(map(chr, resposta.content.upper())).replace(' ', ''))
                        #tramitacao_encerrada = re.search('FIMTRAMITA&CCEDIL;&ATILDE;O:(.+?)\t\t\t\t\t\t', retorno).group(1)
                        if(re.search('FIMTRAMITA&CCEDIL;&ATILDE;O:SIM', retorno)):
                            tramitacao_encerrada = 'SIM'
                        else:
                            tramitacao_encerrada = 'NAO'
                    except:
                        tramitacao_encerrada = 'Informacao nao encontrada'
                        notfound_count_tram+=1


                    #Identificacao da aprovacao do projeto
                    if 'Aprovad' in listagem['Situacao'][i] or 'Trasformado em Norma Jurídica' in listagem['Situacao'][i]:
                        aprovado = 'SIM'
                    elif 'Retirad' in listagem['Situacao'][i] or 'Rejeitad' in listagem['Situacao'][i] or 'Vetado' in listagem['Situacao'][i]:# era 'Vetado Totalmente'\
                        aprovado = 'NAO'
                    elif tramitacao_encerrada == 'NAO': 
                        aprovado = 'TRAMITACAO'
                    #elif listagem['Codigo_Tipo'][i] not in ('REQ', 'MOC', 'REP', 'PEDIF'):
                    #    if listagem['Situacao'][i] == 'Arquivada': 
                    #        aprovado = 'NAO'        
                    #    else:        
                    #        aprovado = 'SIM'
                    #elif listagem['Codigo_Tipo'][i] in ('REQ', 'MOC', 'REP', 'PEDIF'): 
                    else:
                        try:
                            # Limpeza para pegar o id do projeto dentro da prefeitura
                            #retorno = (''.join(map(chr, resposta.content.upper())).replace(' ', ''))

                            if(re.search('APROVAD', retorno)):
                                aprovado = 'SIM'
                            else:
                                aprovado = 'NAO'
                        except:
                            aprovado = 'Informacao nao encontrada'
                            notfound_count_aprv += 1

                except:
                    failed = True
                    time.sleep(180)

                listagem['Data_Projeto'][i] = data_projeto
                listagem['Tramitacao_Encerrada'][i] = tramitacao_encerrada
                listagem['Aprovado'][i] = aprovado

                resposta.close()
                session.close()
                time.sleep(7.5)

        if failed:
            failed_req += 1
        else:
            filed_req = 0
                
#       Se houve 5 falhas consecutivas
        if failed_req > max_failed_req:
            break
            
    if failed_req > max_failed_req:
        break               
            
    count = 0          

    for i in listagem.index:
        if listagem['Data_Projeto'][i] == '' and listagem['Id_Projeto'][i] != '':
            count +=1

if failed_req > max_failed_req:
    print(f'O endpoint nao esta respondendo, verifique a conexao. Fim do processo. Nao encontrados = {notfound_count}')
else:
    print(f'Fim do processo. Nao encontrados = {notfound_count}')

### Antigo

In [ ]:
# Preenchimento do campo Id_Projeto
count = 1
failed_req= 0
failed = False
id_projeto = ''
max_failed_req = 5
notfound_count = 0

while count > 0:
    for i in listagem.index:
        if listagem['Id_Projeto'][i] == '' and listagem['Codigo_Tipo'][i] not in ('DESCONHECIDO','EMENDA'): 
            id_projeto = ''
            failed = False
            # Variaveis de requisicao
            num_proj = int(listagem['Projeto'][i])
            ano_proj = int(listagem['Ano'][i])
            tipo_proj = listagem['Codigo_Tipo'][i]

            try:
                # Requisicao para o portal de propostas da Camara
                query = {"numero": num_proj, "ano": ano_proj, "codtipo": tipo_proj}
                #query = {"numero": 193600, "ano": 2012, "tipo": 'REQ'}
                resposta = requests.get('https://www.camarajf.mg.gov.br/sal/listapropos.php?', params=query)
                try:
                    # Limpeza para pegar o id do projeto dentro da prefeitura
                    retorno = (''.join(map(chr, resposta.content.upper())).replace(' ', ''))
                    id_projeto = re.search('PROPOSICAO.PHP(.+?)TARGET=BLANK', retorno).group(1).replace('?NUM=','').replace('"','')
                except:
                    try:
                        retorno = (''.join(map(chr, resposta.content.upper())).replace(' ', ''))
                        id_projeto = re.search('PROPOSICAON.PHP(.+?)TARGET=BLANK', retorno).group(1).replace('?NUM=','').replace('"','')
                    except:
                        id_projeto = 'Nao Encontrado'
                        notfound_count += 1
            except:
                #Assinala que houve falha
                failed = True
                time.sleep(180)
                    
            listagem['Id_Projeto'][i] = id_projeto
            resposta.close()
            time.sleep(7.5)
            
        if failed:
            failed_req += 1
        else:
            filed_req = 0
                
#       Se houve 5 falhas consecutivas
        if failed_req > max_failed_req:
            break

    if failed_req > max_failed_req:
        break   
        
    count = 0
    
    for i in listagem.index:
        if listagem['Id_Projeto'][i] == '' and listagem['Codigo_Tipo'][i] not in ('DESCONHECIDO','EMENDA'):
            count +=1

if failed_req > max_failed_req:
    print(f'O endpoint nao esta respondendo, verifique a conexao. Fim do processo. Nao encontrados = {notfound_count}')
else:
    print('Fim do processo')

In [ ]:
# Preenchimento do campo Data_Projeto, Tramitacao_Encerrada e Aprovado
count = 1
failed_req= 0
failed = False
max_failed_req = 5
notfound_count = 0

data_projeto = ''
notfound_count_dt = 0

tramitacao_encerrada = ''
notfound_count_tram = 0

aprovado = ''
notfound_count_aprv = 0

while count > 0:
    for i in listagem.index:
        if (listagem['Data_Projeto'][i] == '' or listagem['Tramitacao_Encerrada'][i] == '' or listagem['Aprovado'][i] == '') and listagem['Id_Projeto'][i] != '': 
            data_projeto = ''
            tramitacao_encerrada = ''
            aprovado = ''

            failed = False
            # Variaveis de requisicao
            id_proj = listagem['Id_Projeto'][i]
            
            try:
                # Requisicao para o portal de propostas da Camara
                query = {"num": id_proj}
                resposta = requests.get('https://www.camarajf.mg.gov.br/sal/proposicao.php?', params=query)
                retorno = (''.join(map(chr, resposta.content.upper())).replace(' ', ''))
                
                #Identificacao da data de registro do projeto
                try:
                    # Limpeza para pegar o id do projeto dentro da prefeitura
                    #retorno = (''.join(map(chr, resposta.content.upper())).replace(' ', ''))
                    data_projeto = re.search('PER&IACUTE;ODO:(.+?)A', retorno).group(1)
                except:
                    data_projeto = 'Data nao encontrada'
                    notfound_count_dt += 1
                
                
                #Identificacao do encerramento da tramitacao do projeto
                try:
                    # Limpeza para pegar o id do projeto dentro da prefeitura
                    #retorno = (''.join(map(chr, resposta.content.upper())).replace(' ', ''))
                    #tramitacao_encerrada = re.search('FIMTRAMITA&CCEDIL;&ATILDE;O:(.+?)\t\t\t\t\t\t', retorno).group(1)
                    if(re.search('FIMTRAMITA&CCEDIL;&ATILDE;O:SIM', retorno)):
                        tramitacao_encerrada = 'SIM'
                    else:
                        tramitacao_encerrada = 'NAO'
                except:
                    tramitacao_encerrada = 'Informacao nao encontrada'
                    notfound_count_tram+=1
                    
                    
                #Identificacao da aprovacao do projeto
                if 'Aprovad' in listagem['Situacao'][i] or 'Trasformado em Norma Jurídica' in listagem['Situacao'][i]:
                    aprovado = 'SIM'
                elif 'Retirad' in listagem['Situacao'][i] or 'Rejeitad' in listagem['Situacao'][i] or 'Vetado' in listagem['Situacao'][i]:# era 'Vetado Totalmente'\
                    aprovado = 'NAO'
                elif tramitacao_encerrada == 'NAO': 
                    aprovado = 'TRAMITACAO'
                #elif listagem['Codigo_Tipo'][i] not in ('REQ', 'MOC', 'REP', 'PEDIF'):
                #    if listagem['Situacao'][i] == 'Arquivada': 
                #        aprovado = 'NAO'        
                #    else:        
                #        aprovado = 'SIM'
                #elif listagem['Codigo_Tipo'][i] in ('REQ', 'MOC', 'REP', 'PEDIF'): 
                else:
                    try:
                        # Limpeza para pegar o id do projeto dentro da prefeitura
                        #retorno = (''.join(map(chr, resposta.content.upper())).replace(' ', ''))

                        if(re.search('APROVAD', retorno)):
                            aprovado = 'SIM'
                        else:
                            aprovado = 'NAO'
                    except:
                        aprovado = 'Informacao nao encontrada'
                        notfound_count_aprv += 1
                
            except:
                failed = True
                time.sleep(180)

            listagem['Data_Projeto'][i] = data_projeto
            listagem['Tramitacao_Encerrada'][i] = tramitacao_encerrada
            listagem['Aprovado'][i] = aprovado
            
            resposta.close()
            time.sleep(7.5)
            
        if failed:
            failed_req += 1
        else:
            filed_req = 0
                
#       Se houve 5 falhas consecutivas
        if failed_req > max_failed_req:
            break
            
    if failed_req > max_failed_req:
        break               
            
    count = 0          

    for i in listagem.index:
        if listagem['Data_Projeto'][i] == '' and listagem['Id_Projeto'][i] != '':
            count +=1

if failed_req > max_failed_req:
    print(f'O endpoint nao esta respondendo, verifique a conexao. Fim do processo. Nao encontrados = {notfound_count}')
else:
    print(f'Fim do processo. Nao encontrados = {notfound_count}')